In [1]:
#import libraries
import keras
from keras.datasets import cifar10
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, Flatten, Dropout, BatchNormalization, Activation, AveragePooling2D
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from math import ceil
import os
from keras.preprocessing.image import ImageDataGenerator

In [2]:
#load the mnist dataset
(train_x, train_y), (test_x, test_y) = cifar10.load_data()

In [3]:
#normalize the data
train_x = train_x.astype('float32')/255
train_y = train_y.astype('float32')/255


In [4]:
train_y.shape, train_x.shape, test_x.shape, test_y.shape

((50000, 1), (50000, 32, 32, 3), (10000, 32, 32, 3), (10000, 1))

In [5]:
#subtract the mean image  from both the train and test set
train_x =  train_x - train_x.mean()
test_x = test_x - test_x.mean()

In [6]:
#divide by the standard deviation
train_x = train_x/train_x.std(axis= 0)
test_x = test_x/ test_x.std(axis = 0)

In [7]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [8]:
#encode the labels to vectors
train_y = keras.utils.to_categorical(train_y, 10)
test_y = keras.utils.to_categorical(test_y, 10)

In [9]:
#define the common unit
def Unit(x, filters):
  out = BatchNormalization()(x)
  out = Activation('relu')(out)
  out = Conv2D(filters= filters, kernel_size= [3,3], strides= [1,1], padding= 'same')(out)

  return out

In [10]:
#define the model
def MiniModel(input_shape):
  images = Input(input_shape)

  net = Unit(images, 64)
  net = Unit(net, 64)
  net = Unit(net, 64)
  net = MaxPooling2D(pool_size= (2,2))(net)


  net = Unit(net, 128)
  net = Unit(net, 128)
  net = Unit(net, 128)
  net = MaxPooling2D(pool_size= (2,2)) (net)

  net = Unit(net, 256)
  net = Unit(net, 256)
  net = Unit(net, 256)

  net = Dropout(0.25)(net)
  net = AveragePooling2D(pool_size= (2,2))(net)
  net = Flatten()(net)
  net = Dense(units= 10, activation= 'softmax')(net)

  model = Model(inputs =  images, outputs = net)

  return model


input_shape = (32,32,3)
model = MiniModel(input_shape)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 batch_normalization (Batch  (None, 32, 32, 3)         12        
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization_1 (Bat  (None, 32, 32, 64)        256       
 chNormalization)                                                
                                                                 
 activation_1 (Activation)   (None, 32, 32, 64)        0     

In [12]:
def lr_schedule(epoch):
  lr = 0.001

  if epoch > 15:
      lr = lr/100
  elif lr> 10:
      lr = lr/10

  elif epoch > 5:
      lr = lr/5

  print ('Learning Rate: ', lr)

  return lr

In [13]:
#pass the scheduler function to the Learning Rate Scheduler Class
lr_scheduler = LearningRateScheduler(lr_schedule)

In [14]:
#directory to create the model
save_directory = os.path.join(os.getcwd(), 'cifar10savedmodels')
#name of model files
model_name = 'cifar10model.{epoch:03d}.h5'

#create Directory if it doesn't exist
if not os.path.isdir(save_directory):
  os.makedirs(save_directory)

#join the directory with the model file
modelpath = os.path.join(save_directory, model_name)

In [15]:
checkpoint = ModelCheckpoint(filepath= modelpath,
                             monitor= 'val_acc',
                             verbose= 1,
                             save_best_only= True,
                             period = 1)

In [16]:
#specify the training components
model.compile(optimizer= Adam(lr_schedule(0)), loss = 'categorical_crossentropy', metrics = ['Accuracy'])

Learning Rate:  0.001


In [17]:
datagen = ImageDataGenerator(rotation_range= 10,
                             width_shift_range= 5. /32,
                             height_shift_range= 5. / 32,
                             horizontal_flip= True)

In [18]:
# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(train_x)

In [19]:
epochs = 20

steps_per_epoch = ceil(50000/128)

In [20]:
#fit the model on the batches generated by datagen.flow()
model.fit_generator(datagen.flow(train_x, train_y, batch_size= 128), validation_data= [test_x, test_y], epochs= epochs, steps_per_epoch= steps_per_epoch, verbose = 1, workers= 4, callbacks= [checkpoint, lr_scheduler])

C:\Users\PC\AppData\Local\Temp\ipykernel_15460\3115652652.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(train_x, train_y, batch_size= 128), validation_data= [test_x, test_y], epochs= epochs, steps_per_epoch= steps_per_epoch, verbose = 1, workers= 4, callbacks= [checkpoint, lr_scheduler])


Learning Rate:  0.001
Epoch 1/20
391/391 [==============================] - 2126s 5s/step - loss: 0.0071 - Accuracy: 0.9977 - val_loss: 45.3936 - val_Accuracy: 0.1000 - lr: 0.0010
Learning Rate:  0.001
Epoch 2/20
391/391 [==============================] - 2104s 5s/step - loss: 0.0000e+00 - Accuracy: 1.0000 - val_loss: 96.8324 - val_Accuracy: 0.1000 - lr: 0.0010
Learning Rate:  0.001
Epoch 3/20
101/391 [======>.......................] - ETA: 23:33 - loss: 0.0000e+00 - Accuracy: 1.0000

In [ ]:
#Eevaluate the accuracy of the test dataset
accuracy = model.evaluate(x = test_x, y = test_y, batch_size = 128)